In [1]:
import copy
import sys
import os
import glob
import time
sys.path.append('../..')

from model import processing

start_time = time.time()
price_dict = processing.import_binance_prices(['DOT', 'ETH', 'SOL', 'BTC'], start_date='Nov 6 2022', days=4)
stop_time = time.time()
print(f'load time: {round(stop_time - start_time, 3)} seconds')  
# we want about .000002 * TVL in volume every block

In [7]:
price_data = [{tkn: price_dict[tkn][i] for tkn in ['DOT', 'ETH', 'SOL', 'BTC']} for i in range(len(price_dict[tkn]))]
print(price_data[0], price_data[1])

{'DOT': 7.03, 'ETH': 1627.28, 'SOL': 36.9, 'BTC': 21300.12} {'DOT': 7.02, 'ETH': 1626.28, 'SOL': 36.84, 'BTC': 21296.04}


In [2]:
from pprint import pprint
import pandas as pd
import random
import sys
from matplotlib import pyplot as plt
sys.path.append('../..')

from model import run
from model import plot_utils as pu
from model import processing
from model.amm.omnipool_amm import OmnipoolState, dynamic_asset_fee, dynamic_lrna_fee, dynamicmult_lrna_fee, dynamicmult_asset_fee, dynamicadd_asset_fee, dynamicadd_lrna_fee
from model.amm import omnipool_amm as oamm

from model.amm.agents import Agent
from model.amm.trade_strategies import omnipool_arbitrage, invest_all, back_and_forth
from model.amm.global_state import GlobalState, fluctuate_prices, historical_prices, cash_out_omnipool

# same seed, same parameters = same simulation result
random.seed(42)

assets = {}
omnipool_assets = ['HDX', 'USD', 'AUSD', 'ETH', 'DOT']

assets = {
    'HDX': {'usd price': 0.005, 'weight': 0.10},
    'USD': {'usd price': 1, 'weight': 0.30},
    'BTC': {'usd price': price_dict['BTC'][0], 'weight': 0.10},
    'ETH': {'usd price': price_dict['ETH'][0], 'weight': 0.10},
    'DOT': {'usd price': price_dict['DOT'][0], 'weight': 0.30},
    'SOL': {'usd price': price_dict['SOL'][0], 'weight': 0.10},
}

lrna_price_usd = 22
initial_omnipool_tvl = 10000000
liquidity = {}
lrna = {}

for tkn, info in assets.items():
    liquidity[tkn] = initial_omnipool_tvl * info['weight'] / info['usd price']
    lrna[tkn] = initial_omnipool_tvl * info['weight'] / lrna_price_usd

initial_state = GlobalState(
    pools={
        'omnipool': OmnipoolState(
            tokens={
                tkn: {'liquidity': liquidity[tkn], 'LRNA': lrna[tkn]} for tkn in assets
            },
            oracles={'fee_raise': 50},  # , 'fee_lower': 7200},
            lrna_fee=dynamicadd_lrna_fee(
                minimum=0.0005,
                amplification=1,
                raise_oracle_name='fee_raise',
                decay=0.00001,
                fee_max=0.1,
            ),
            asset_fee=dynamicadd_asset_fee(
                minimum=0.0025,
                amplification=0.2,
                raise_oracle_name='fee_raise',
                decay=0.00005,
                fee_max=0.4,
            ),
        )
    },
    agents = {
        'Arbitrageur': Agent(
            holdings={'HDX': 1000000000, 'USD': 1000000000, 'R1': 1000000000, 'R2': 3000000000, 'DOT': 1000000000, 'LRNA': 1000000000, 'AUSD': 1000000000, 'ETH': 1000000000, 'SOL': 1000000000, 'BTC': 1000000000},
            trade_strategy=omnipool_arbitrage(
                pool_id='omnipool',
                arb_precision=3,
                # skip_assets=['HDX']
            )
        ),
        'Trader': Agent(
            holdings={'HDX': 1000000000, 'USD': 1000000000, 'R1': 1000000000, 'R2': 3000000000, 'DOT': 1000000000, 'LRNA': 1000000000, 'AUSD': 1000000000, 'ETH': 1000000000, 'SOL': 1000000000, 'BTC': 1000000000},
            trade_strategy=back_and_forth(
                pool_id='omnipool',
                percentage=0.000014
            )
        ),
        'LP': Agent(
            holdings={tkn: liquidity[tkn] / 10 for tkn in assets},
            trade_strategy=invest_all('omnipool')
        ),
        'LPDOT': Agent(
            holdings={'DOT': liquidity['DOT'] / 10},
            trade_strategy=invest_all('omnipool')
        ),
        'LPUSD': Agent(
            holdings={'USD': liquidity['USD'] / 10},
            trade_strategy=invest_all('omnipool')
        ),
        'LPETH': Agent(
            holdings={'ETH': liquidity['ETH'] / 10},
            trade_strategy=invest_all('omnipool')
        )
    },
    evolve_function=historical_prices(price_list),
    external_market={tkn: assets[tkn]['usd price'] for tkn in assets},
    archive_all=False,
)

# import cProfile, pstats
# profiler = cProfile.Profile()
# profiler.enable()

time_steps = len(price_dict['ETH'])-1
print(f'Running with {time_steps} time steps')
events = run.run(initial_state, time_steps=time_steps)
# events = processing.postprocessing(events, optional_params=['withdraw_val'])

# profiler.disable()
# stats = pstats.Stats(profiler)
# stats.dump_stats('profile-data2')

AttributeError: 'OmnipoolState' object has no attribute 'last_fee'

In [ ]:
print(type(events[0].pools))

In [ ]:

usd_index = -1
for i in range(len(initial_state.pools['omnipool'].asset_list)):
    if initial_state.pools['omnipool'].asset_list[i] == "USD":
        usd_index = i
        break

# get USD fees
USD_fees = [None]*len(events)
USD_lrna_fees = [None]*len(events)
for i in range(len(events)):
    op = events[i].pools['omnipool']
    USD_fees[i] = op.last_fee['USD']
    USD_lrna_fees[i] = op.last_lrna_fee['USD']


for tkn in initial_state.pools['omnipool'].asset_list:
    prices = [None]*len(events)
    last_fees = [None]*len(events)
    last_lrna_fees = [None]*len(events)
    buy_prices = [None]*len(events)
    sell_prices = [None]*len(events)
    market_prices = [None]*len(events)
    for i in range(len(events)):
        op = events[i].pools['omnipool']
        market_prices[i] = events[i].external_market[tkn]
        prices[i] = oamm.usd_price(op, tkn)
        last_fees[i] = op.last_fee[tkn]
        last_lrna_fees[i] = op.last_lrna_fee[tkn]
        buy_prices[i] = prices[i] / (1 - last_fees[i]) / (1 - USD_lrna_fees[i])
        sell_prices[i] = prices[i] * (1 - USD_fees[i]) * (1 - last_lrna_fees[i])

    ax = pu.plot(prices)
    ax = pu.plot(buy_prices, subplot=ax)
    ax = pu.plot(sell_prices, subplot=ax)
    # pprint(tkn)
    pu.plot(market_prices, subplot=ax)
    plt.title(tkn)


In [ ]:
market = events[0].external_market
print(market)

In [ ]:

print(":::FEES:::")

for tkn in initial_state.pools['omnipool'].asset_list:
    last_fee = [events[i].pools['omnipool'].last_fee[tkn] for i in range(100,len(events))]
    pu.plot(last_fee)
    plt.title(tkn)

In [ ]:
for tkn in initial_state.pools['omnipool'].asset_list:
    last_lrna_fee = [events[i].pools['omnipool'].last_lrna_fee[tkn] for i in range(100,len(events))]
    pu.plot(last_fee)
    plt.title(tkn)

In [ ]:
basic_state = initial_state.copy()
basic_state.pools['omnipool'].lrna_fee = 0.0005
basic_state.pools['omnipool'].asset_fee = 0.0025

events2 = run.run(basic_state, time_steps=time_steps)
# events2 = processing.postprocessing(events2, optional_params=['withdraw_val'])

# print(events2[-1])
# feeless_arbitrageur = events2[-1]['state'].agents['Arbitrageur']

In [ ]:
lrna_prices = [1/oamm.price(event.pools['omnipool'], "USD") for event in events[100:]]
lrna_prices2 = [1/oamm.price(event.pools['omnipool'], "USD") for event in events2[100:]]
ax = pu.plot(lrna_prices)
pu.plot(lrna_prices2, subplot=ax)

In [ ]:
usd_index = -1
for i in range(len(initial_state.pools['omnipool'].asset_list)):
    if initial_state.pools['omnipool'].asset_list[i] == "USD":
        usd_index = i
        break

# get USD fees
USD_fees = [None]*len(events2)
USD_lrna_fees = [None]*len(events2)
for i in range(len(events2)):
    op = events2[i].pools['omnipool']
    USD_fees[i] = op.last_fee['USD']
    USD_lrna_fees[i] = op.last_lrna_fee['USD']


for tkn in initial_state.pools['omnipool'].asset_list:
    prices = [None]*len(events2)
    last_fees = [None]*len(events2)
    last_lrna_fees = [None]*len(events2)
    buy_prices = [None]*len(events2)
    sell_prices = [None]*len(events2)
    market_prices = [None]*len(events2)
    for i in range(len(events2)):
        op = events2[i].pools['omnipool']
        market_prices[i] = events2[i].external_market[tkn]
        prices[i] = oamm.usd_price(op, tkn)
        last_fees[i] = op.last_fee[tkn]
        last_lrna_fees[i] = op.last_lrna_fee[tkn]
        buy_prices[i] = prices[i] / (1 - last_fees[i]) / (1 - USD_lrna_fees[i])
        sell_prices[i] = prices[i] * (1 - USD_fees[i]) * (1 - last_lrna_fees[i])

    ax = pu.plot(prices)
    ax = pu.plot(buy_prices, subplot=ax)
    ax = pu.plot(sell_prices, subplot=ax)
    pu.plot(market_prices, subplot=ax)
    plt.title(tkn)


In [ ]:
basic_state = initial_state.copy()
basic_state.pools['omnipool'].lrna_fee = 0.0000
basic_state.pools['omnipool'].asset_fee = 0.0000

events3 = run.run(basic_state, time_steps=time_steps)

In [ ]:
usd_index = -1
for i in range(len(initial_state.pools['omnipool'].asset_list)):
    if initial_state.pools['omnipool'].asset_list[i] == "USD":
        usd_index = i
        break

# get USD fees
USD_fees = [None]*len(events3)
USD_lrna_fees = [None]*len(events3)
for i in range(len(events3)):
    op = events3[i].pools['omnipool']
    USD_fees[i] = op.last_fee['USD']
    USD_lrna_fees[i] = op.last_lrna_fee['USD']


for tkn in initial_state.pools['omnipool'].asset_list:
    prices = [None]*len(events3)
    last_fees = [None]*len(events3)
    last_lrna_fees = [None]*len(events3)
    buy_prices = [None]*len(events3)
    sell_prices = [None]*len(events3)
    market_prices = [None]*len(events3)
    for i in range(len(events3)):
        op = events3[i].pools['omnipool']
        market_prices[i] = events3[i].external_market[tkn]
        prices[i] = oamm.usd_price(op, tkn)
        last_fees[i] = op.last_fee[tkn]
        last_lrna_fees[i] = op.last_lrna_fee[tkn]
        buy_prices[i] = prices[i] / (1 - last_fees[i]) / (1 - USD_lrna_fees[i])
        sell_prices[i] = prices[i] * (1 - USD_fees[i]) * (1 - last_lrna_fees[i])

    ax = pu.plot(prices)
    ax = pu.plot(buy_prices, subplot=ax)
    ax = pu.plot(sell_prices, subplot=ax)
    pu.plot(market_prices, subplot=ax)
    plt.title(tkn)


In [ ]:
LP_AMT = {tkn: liquidity[tkn]/10 for tkn in ['ETH', 'DOT', 'USD']}

LP_cashout_dynamic = {}
LP_cashout_static = {}
LP_cashout_zero = {}
LP_profit_dynamic = {}
LP_profit_static = {}
LP_profit_zero = {}
LP_HOLD = {}

real_market_prices = [e.external_market for e in events]
n = len(events)

for tkn in ['ETH', 'DOT', 'USD']:
    LP_HOLD[tkn] = [LP_AMT[tkn] * p_dict[tkn] for p_dict in real_market_prices]

    LP_cashout_dynamic[tkn] = [cash_out_omnipool(e.pools['omnipool'], e.agents['LP' + tkn], e.external_market) for e in events]
    LP_cashout_static[tkn] = [cash_out_omnipool(e.pools['omnipool'], e.agents['LP' + tkn], e.external_market) for e in events2]
    LP_cashout_zero[tkn] = [cash_out_omnipool(e.pools['omnipool'], e.agents['LP' + tkn], e.external_market) for e in events3]

    LP_profit_dynamic[tkn] = [(LP_cashout_dynamic[tkn][i] - LP_HOLD[tkn][i])/LP_HOLD[tkn][i] for i in range(n)]
    LP_profit_static[tkn] = [(LP_cashout_static[tkn][i] - LP_HOLD[tkn][i])/LP_HOLD[tkn][i] for i in range(n)]
    LP_profit_zero[tkn] = [(LP_cashout_zero[tkn][i] - LP_HOLD[tkn][i])/LP_HOLD[tkn][i] for i in range(n)]

    ax = pu.plot(LP_profit_dynamic[tkn])
    ax = pu.plot(LP_profit_static[tkn], subplot=ax)
    pu.plot(LP_profit_zero[tkn], subplot=ax)
    plt.title(tkn)